In [ ]:
class myconfig():
    original_data = "../data/mini_dev.pkl"
    
    train_data= "../data/minidev_train.pkl"
    test_data= "../data/minidev_case1.pkl"
    unseen_data= "../data/minidev_case2.pkl"
    
    jsonl_data_train= "../data/mini_dataset_train.jsonl"
    jsonl_data_test= "../data/mini_dataset_case_1.jsonl"
    jsonl_data_unseen= "../data/mini_dataset_case_2.jsonl"
mcfg = myconfig()

# 데이터 확인

In [ ]:
import pandas as pd
import ast

df = pd.read_pickle(mcfg.original_data)
df

In [ ]:
df["db_id"].value_counts()

# unseen / train / test 분할

In [ ]:
df_unseen = df[(df["db_id"] == "financial")]
df_unseen["db_id"].value_counts()

In [ ]:
df_seen = df[~df.isin(df_unseen).all(axis=1)]
df_seen["db_id"].value_counts()

In [ ]:
# train - test
from sklearn.model_selection import train_test_split

df_train = pd.DataFrame(columns=df_seen.columns)
df_test = pd.DataFrame(columns=df_seen.columns)

db_list = list(set(df_seen["db_id"].values))

for db in db_list:
    sample_df = df_seen[df_seen["db_id"] == db]
    sample_df_train, sample_df_test = train_test_split(sample_df, test_size=0.1, stratify=sample_df["difficulty"], random_state=42)

    df_train = pd.concat([df_train,sample_df_train], axis=0, ignore_index=True)
    df_test = pd.concat([df_test,sample_df_test], axis=0, ignore_index=True)

In [ ]:
print(len(df_train))
print(len(df_test))
print(len(df_unseen))

In [ ]:
df_train.to_pickle(mcfg.train_data)
df_test.to_pickle(mcfg.test_data)
df_unseen.to_pickle(mcfg.unseen_data)

# df to json

In [ ]:
import pandas as pd
df_train = pd.read_pickle(mcfg.train_data)
df_test = pd.read_pickle(mcfg.test_data)
df_unseen = pd.read_pickle(mcfg.unseen_data)

In [ ]:
df_train

# train

In [ ]:
import json
with open(mcfg.jsonl_data_train, 'w', encoding='utf-8') as f:
    for i, data in df_train.iterrows():
        messages = {"messages":[{"role":"system", "content":data["instruction"]},
                                {"role":"user", "content":data["input"]},
                                {"role":"assistant", "content":data["output"]}
                                ]
                    }
        f.write(json.dumps(messages, ensure_ascii=False) + '\n')

In [ ]:
a = []
with open(mcfg.jsonl_data_train, 'r', encoding='utf-8') as f:
    for line in f:
        a.append(line)

In [ ]:
len(a)

# test

In [ ]:
import json
with open(mcfg.jsonl_data_test, 'w', encoding='utf-8') as f:
    for i, data in df_test.iterrows():
        messages = {"messages":[{"role":"system", "content":data["instruction"]},
                                {"role":"user", "content":data["input"]},
                                ]
                    }
        f.write(json.dumps(messages, ensure_ascii=False) + '\n')

In [ ]:
a = []
with open(mcfg.jsonl_data_test, 'r', encoding='utf-8') as f:
    for line in f:
        a.append(line)

In [ ]:
len(a)

# unseen

In [ ]:
import json
with open(mcfg.jsonl_data_unseen, 'w', encoding='utf-8') as f:
    for i, data in df_unseen.iterrows():
        messages = {"messages":[{"role":"system", "content":data["instruction"]},
                                {"role":"user", "content":data["input"]},
                                ]
                    }
        f.write(json.dumps(messages, ensure_ascii=False) + '\n')

In [ ]:
a = []
with open(mcfg.jsonl_data_unseen, 'r', encoding='utf-8') as f:
    for line in f:
        a.append(line)

In [ ]:
len(a)